# Geodetic motion in Schwarzschild geometry

### The metric and the geodesic equation

The metric of the Schwarzschild space around a not charged, not spinnig black hole is $$ ds^2 = -(1-\frac{2GM}{r})c^2 dt^2 +(1-\frac{2GM}{r})^{-1} dr^2 + r^2 d\theta^2 + r^2 \sin(\theta)^2 d\phi^2 $$ 
The metric is described by $ g_{\mu \nu} = diag(-(1-\frac{2GM}{r}), (1-\frac{2GM}{r})^{-1}, r^2, r^2\sin(\theta)^2) $ 
 
The evolution of a massive body in geodesic motion is described by
$$ \begin{equation}
    \begin{cases}
      \frac{dx^\mu}{d \tau}= u^\mu\\
      \frac{du^\alpha}{d \tau} + \Gamma^\alpha_{\beta \gamma}u^\beta u^\gamma = 0
    \end{cases}\
\end{equation} $$
The equation can be integrated in the proper time to obtain the coordinate time and the position of the body $ x^\mu = (t, r, \theta, \phi) $ 
 
  
The non-zero Christoffel symbols are 

$$ \begin{matrix}
 \Gamma^r_{tt}= \frac{GM(r-2GM)}{r^3}  &  \Gamma^r_{\phi\phi}= -(r-2GM)\sin2\theta  &   \Gamma^r_{\theta\theta}= -(r-2GM)  \\
 \Gamma^r_{rr}= -GMr(r-2GM)  &  \Gamma^t_{rt}= \frac{GM}{r(r-2GM)}  &   \Gamma^\theta_{\phi\phi}= -\sin\theta\cos\theta  \\
 \Gamma^\phi_{r\phi}= \frac{1}{r}  &   \Gamma^\theta_{r\theta}= \frac{1}{r}  &   \Gamma^\phi_{\theta\phi}=\sin\theta\cos\theta 
\end{matrix} $$

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ani
import pandas as pd
from scipy.integrate import solve_ivp

### Definitions of the metric and the Christoffel symbols

In [26]:
# The metric depends on the point where it is calculated

g = lambda x, Rs: np.diag([-(1-2*Rs/x[1]),(1-2*Rs/x[1])**-1, x[1]**2, (x[1]*np.sin(x[2]))**2])

# Christoffel symbols

def Chr(x, Rs):
  r, theta = x[1], x[2]
  ch = np.zeros((4,4,4))
  ch[1][0][0] = Rs*(r-2*Rs)/r**3      # r-tt
  ch[0][1][0] = Rs/(r*(r-2*Rs))    # t-rt
  ch[0][0][1] = Rs/(r*(r-2*Rs))    # t-tr
  
  ch[1][1][1] = -Rs/(r*(r-2*Rs))    # r-rr
  
  ch[1][2][2] = -(r-2*Rs)     # r-theta theta
  ch[2][1][2] = -(r-2*Rs)     # theta - r theta
  ch[2][2][1] = -(r-2*Rs)     # theta - theta r
  
  ch[1][3][3] = -(r-2*Rs)*np.sin(2*theta)    # r- phi phi
  ch[3][1][3] = 1/r    # phi- r phi
  ch[3][3][1] = 1/r    # phi- phi r
  
  ch[2][3][3] = -0.5*np.sin(2*theta)   # theta - phi phi
  ch[3][2][3] = 0.5*np.sin(2*theta)   # phi - theta phi
  ch[3][3][2] = 0.5*np.sin(2*theta)   # phi - phi theta
  
  return ch

def dot(a,b,x): #Doct product with the metric
  return np.dot(a,np.dot(g(x), b))

def spher2cart(x):  #Spherical to cartesian coordinates
  r, theta, phi = x[0], x[1], x[2]
  return (r*np.sin(theta)*np.cos(phi), r*np.sin(theta)*np.sin(phi), r*np.cos(theta))
  

In [27]:
G, M, c = 1.0, 1.0, 1.0
Rs = G*M/c**2

print(f'The Schwarzschild radius is {Rs:.3e}')

The Schwarzschild radius is 1.000e+00


In [28]:
#Initial position and speed

x = (10, 0.5*np.pi, 0)  # (r, theta, phi)
v = (0,0,1e-4)          # (v_r, v_theta, v_phi) in c units

x_cart = spher2cart(x)
print(f'The initial position is (r: {x[0]:.3e}, theta: {x[1]:.3e}, phi: {x[2]:.3e}) or (x: {x_cart[0]:.3e}, y: {x_cart[1]:.3e}, z: {x_cart[2]:.3e})')

dx = (v[0], v[1]/x[0], v[2]/(x[0]*np.sin(x[1]))) #dx/dt: dr = v_r, dtheta = v_theta/r, dphi = v_phi/(r*sin(theta))
print(f'The initial velocity is (v_r: {v[0]:.3e}, v_theta: {v[1]:.3e}, v_phi: {v[2]:.3e}) or (dr/dt: {dx[0]:.3e}, dtheta/dt: {dx[1]:.3e}, dphi/dt: {dx[2]:.3e})')
print(f'The total velocity is {np.linalg.norm(v):.3e} c')

#Initial 4-vectors
x = np.array([0, x[0], x[1], x[2]])

# u is found by using u*u = -1 and u0 = dt/dtau

The initial position is (r: 1.000e+01, theta: 1.571e+00, phi: 0.000e+00) or (x: 1.000e+01, y: 0.000e+00, z: 6.123e-16)
The initial velocity is (v_r: 0.000e+00, v_theta: 0.000e+00, v_phi: 1.000e-04) or (dr/dt: 0.000e+00, dtheta/dt: 0.000e+00, dphi/dt: 1.000e-05)
The total velocity is 1.000e-04 c
